# Lasso Regression - Firm Characteristics

In [40]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

Data

In [2]:
df = pd.read_csv("economic_story.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,AVY.N,2013-01-01,0.54,0.48571,11.178,Materials,3.625620,3.426001e+09,1.532200e+09,4.413001e+09,...,3.00000,3.000,33400000.0,36.00000,2.44949,34.92,0.799347,10.637479,207295.0,11.178
1,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,5.459128e+09,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.895787,24.148726,0.0,2.482
2,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0,1.068
3,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0,8.095
4,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0,1.471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,1.769954e+10,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,3.684400,-5.130322,157740.0,17.194
20116,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,2.384882e+10,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,4.504382,30.489705,82349.0,40.267
20117,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,1.843682e+10,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,3.422880,-25.160664,290897.0,34.342
20118,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,1.556187e+10,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,2.749481,-21.661570,185584.0,1.503


In [3]:
df.head(10)

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,AVY.N,2013-01-01,0.54,0.48571,11.178,Materials,3.625620,3.426001e+09,1.532200e+09,4.413001e+09,...,3.00000,3.00000,33400000.0,36.00000,2.44949,34.92,0.799347,10.637479,207295.0,11.178
1,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,5.459128e+09,...,2.87500,3.00000,66800000.0,42.57143,3.92272,43.07,0.895787,24.148726,0.0,2.482
2,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,...,2.87500,2.87500,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0,1.068
3,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,...,2.62500,2.87500,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0,8.095
4,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,...,2.33333,2.62500,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0,1.471
5,AVY.N,2014-04-01,0.65,0.66129,-1.707,Materials,2.068684,4.796814e+09,1.550100e+09,5.710014e+09,...,2.37500,2.33333,71600000.0,55.14286,2.89968,50.67,0.902107,1.549025,176070.0,1.707
6,AVY.N,2014-07-01,0.80,0.79429,0.719,Materials,1.138123,4.901146e+09,1.615800e+09,5.851946e+09,...,2.25000,2.37500,44400000.0,56.00000,2.72554,51.25,0.935594,3.259666,214940.0,0.719
7,AVY.N,2014-10-01,0.77,0.73700,4.478,Materials,2.126187,4.350009e+09,1.559600e+09,5.266609e+09,...,2.50000,2.25000,60900000.0,52.57143,3.33197,44.65,0.821118,-12.244500,363322.0,4.478
8,AVY.N,2015-01-01,0.90,0.80414,11.921,Materials,0.917751,4.718123e+09,1.604800e+09,5.655323e+09,...,2.12500,2.50000,69800000.0,56.64286,4.42972,51.88,0.894368,17.020360,119821.0,11.921
9,AVY.N,2015-04-01,0.81,0.69414,16.691,Materials,3.512260,4.794717e+09,1.528000e+09,5.793217e+09,...,2.25000,2.12500,71900000.0,58.50000,1.80278,52.91,0.904187,2.647225,210533.0,16.691


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
df.nunique()

Instrument                                               503
Date                                                      41
Earnings Per Share - Actual                             1699
Earnings Per Share - Mean                              17516
Earnings Per Share - Actual Surprise                   14743
GICS Industry Group Name                                  24
Earnings Per Share – Coefficient of Variation          19305
Market Capitalization                                  19426
Revenue - Actual                                       16270
Enterprise Value                                       19424
Number of Analysts                                        50
Recommendation - Mean (1-5)                              932
Recommendation - Mean (1-5).1                            930
Net Income after Tax                                   12176
Price Target - Mean                                    14924
Price Target - Standard Deviation                      17397
Price Close             

Remove outliers

In [6]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,AVY.N,2013-01-01,0.54,0.48571,11.178,Materials,3.625620,3.426001e+09,1.532200e+09,4.413001e+09,...,3.00000,3.000,33400000.0,36.00000,2.44949,34.92,0.799347,10.637479,207295.0,11.178
1,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,5.459128e+09,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.895787,24.148726,0.0,2.482
2,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0,1.068
3,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0,8.095
4,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0,1.471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,1.769954e+10,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,3.684400,-5.130322,157740.0,17.194
20116,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,2.384882e+10,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,4.504382,30.489705,82349.0,40.267
20117,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,1.843682e+10,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,3.422880,-25.160664,290897.0,34.342
20118,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,1.556187e+10,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,2.749481,-21.661570,185584.0,1.503


In [44]:
cols_to_use = ['Market Capitalization', 'Revenue - Actual', 
               'Recommendation - Mean (1-5)', '3 Month Total Return', 'Earnings Per Share - Actual Surprise AbsVals' ]

df_lasso = df[cols_to_use]


In [45]:
df_lasso.head()

,Market Capitalization,Revenue - Actual,Recommendation - Mean (1-5),3 Month Total Return,Earnings Per Share - Actual Surprise AbsVals
0,3.426001e+09,1.532200e+09,3.00000,10.637479,11.178
1,4.309428e+09,1.498900e+09,2.87500,24.148726,2.482
2,4.258229e+09,1.552300e+09,2.87500,-0.054218,1.068
3,4.273414e+09,1.504900e+09,2.62500,2.467620,8.095
4,4.877168e+09,1.583900e+09,2.33333,16.003852,1.471


In [46]:
df_lasso.isna().sum()

Market Capitalization                            600
Revenue - Actual                                 708
Recommendation - Mean (1-5)                     1471
3 Month Total Return                             597
Earnings Per Share - Actual Surprise AbsVals     689
dtype: int64

In [47]:
cols_to_fill_zero = ['Recommendation - Mean (1-5)']
df_lasso[cols_to_fill_zero] = df_lasso[cols_to_fill_zero].fillna(0)
df_lasso.isna().sum()

Market Capitalization                           600
Revenue - Actual                                708
Recommendation - Mean (1-5)                       0
3 Month Total Return                            597
Earnings Per Share - Actual Surprise AbsVals    689
dtype: int64

In [48]:
df_lasso.dropna(inplace=True)
df_lasso.isna().sum()

Market Capitalization                           0
Revenue - Actual                                0
Recommendation - Mean (1-5)                     0
3 Month Total Return                            0
Earnings Per Share - Actual Surprise AbsVals    0
dtype: int64

In [49]:
df_lasso = pd.get_dummies(df_lasso, drop_first=True)
df_lasso

,Market Capitalization,Revenue - Actual,Recommendation - Mean (1-5),3 Month Total Return,Earnings Per Share - Actual Surprise AbsVals
0,3.426001e+09,1.532200e+09,3.00000,10.637479,11.178
1,4.309428e+09,1.498900e+09,2.87500,24.148726,2.482
2,4.258229e+09,1.552300e+09,2.87500,-0.054218,1.068
3,4.273414e+09,1.504900e+09,2.62500,2.467620,8.095
4,4.877168e+09,1.583900e+09,2.33333,16.003852,1.471
...,...,...,...,...,...
20115,1.742020e+10,1.411448e+09,2.30000,-5.130322,17.194
20116,2.268979e+10,1.035557e+09,2.30000,30.489705,40.267
20117,1.696712e+10,1.412650e+09,2.00000,-25.160664,34.342
20118,1.405795e+10,2.055818e+09,2.20000,-21.661570,1.503


In [50]:
rescale = df_lasso
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5)"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5)"]).reshape(-1,1))
df_lasso = rescale
df_lasso

,Market Capitalization,Revenue - Actual,Recommendation - Mean (1-5),3 Month Total Return,Earnings Per Share - Actual Surprise AbsVals
0,0.001166,0.065123,0.812499,0.298034,11.178
1,0.001471,0.064917,0.778645,0.340793,2.482
2,0.001453,0.065247,0.778645,0.264198,1.068
3,0.001459,0.064955,0.710937,0.272179,8.095
4,0.001667,0.065443,0.631943,0.315017,1.471
...,...,...,...,...,...
20115,0.006004,0.064377,0.622916,0.248134,17.194
20116,0.007826,0.062055,0.622916,0.360861,40.267
20117,0.005848,0.064385,0.541666,0.184744,34.342
20118,0.004842,0.068358,0.595833,0.195817,1.503


In [51]:
X = df_lasso.drop('Earnings Per Share - Actual Surprise AbsVals', axis=1)
y = df_lasso['Earnings Per Share - Actual Surprise AbsVals']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

In [53]:
lr.score(X_test, y_test)

-0.025303005554272007

In [54]:
lr.score(X_train, y_train)

2.42311516865934e-05

In [55]:
lasso_reg = linear_model.Lasso(alpha = 50, max_iter = 100, tol=0.1)

lasso_reg.fit(X_train, y_train)

Lasso(alpha=50, max_iter=100, tol=0.1)

In [56]:
lasso_reg.score(X_test, y_test)

-0.016732669958622992

In [57]:
lasso_reg.score(X_train, y_train)

0.0

Standardising

In [ ]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

In [ ]:
df_lasso = df_clean.copy()

In [ ]:
df_clean

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [ ]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

In [ ]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


In [ ]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary
